# Import data and EDA


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

cities = pd.read_csv("CHDB_data_city_all v7_1.csv")
print("Dataset dimensions: " + str(cities.shape))
# Take a look at the number of unique values for each column
for col_name in cities.columns:
    column = cities[col_name]
    print(col_name + ": "  + str(column.nunique()))

Dataset dimensions: (60500, 22)
state_abbr: 51
state_fips: 51
place_fips: 397
stpl_fips: 500
city_name: 474
metric_name: 36
group_name: 14
metric_number: 36
group_number: 14
num: 9469
denom: 13957
est: 3217
lci: 2839
uci: 3133
county_indicator: 4
educ_indicator: 3
multiplier_indicator: 2
data_yr_type: 12
geo_level: 1
date_export: 1
version: 1
NOTE - NCHS Disclaimer: 1


# 1. Generate rankings for 500 cities, print where Flint is in bottom 3%

In [68]:
# eliminate all extraneous columns (year of collection, state code, etc.)
metrics = cities[['metric_name', 'num', 'denom', 'est']]

# create descrtive statistics for all cities
overall_avg = metrics.groupby(['metric_name']).mean()

total_pop = cities[cities['group_name'] == 'total population']
overall_metrics = total_pop[['city_name', 'metric_name', 'est']]

print("Categories where Flint is in the bottom 3%: \n")
print("Out of 500 cities, Flint is ranked...")
for metric in overall_metrics.metric_name.unique():
    metric_df = overall_metrics[overall_metrics['metric_name'] == metric]
    sorted_df = metric_df.sort_values(['est'])
    sorted_df = sorted_df.reset_index(drop=True)
    flint_rank = np.where(sorted_df['city_name'] == 'Flint')[0]
    flint_rank += 1
    if flint_rank < 30:
        flint_rank = abs(flint_rank - 500)
    if int(str(flint_rank)[1:-1]) < 30 or int(str(flint_rank)[1:-1]) > 470:
        print(str(flint_rank)[1:-1] + ' in ' + metric)

Categories where Flint is in the bottom 3%: 

Out of 500 cities, Flint is ranked...
500 in Children in Poverty
480 in Dental care
495 in Diabetes
500 in Frequent mental distress
500 in Frequent physical distress
488 in High blood pressure
499 in Income Inequality
482 in Lead exposure risk index
477 in Neighborhood racial/ethnic segregation
498 in Obesity
476 in Physical inactivity
488 in Preventive services
500 in Smoking
481 in Third-grade reading proficiency
500 in Unemployment
476 in Cardiovascular disease deaths
477 in Colorectal cancer deaths
487 in Premature deaths (all causes)
496 in Life expectancy


In [67]:
# create dataset specificalle for flint
flint_total = cities[(cities['city_name'] == 'Flint') & (cities['group_name'] == 'total population')]
flint_vals = flint_total[['metric_name', 'est']]
flint_vals.set_index('metric_name', inplace=True)
#drop gender/ethnic subcategories, keep total population
metrics_totalpop = cities[cities['group_name'] == 'total population']
overall_stats = metrics_totalpop[['metric_name', 'est']].groupby(['metric_name']).describe()

# in what categories is Flint an outlier?
# calculate dc's z-score for each metrics
overall_stats.columns = overall_stats.columns.get_level_values(1)
overall_stats['flint_avg'] = flint_vals['est']
overall_stats['flint_z-score'] = (overall_stats['flint_avg'] - overall_stats['mean']) / overall_stats['std']
print("Stats for all 500 cities:")
overall_stats

Stats for all 500 cities:


,count,mean,std,min,25%,50%,75%,max,flint_avg,flint_z-score
metric_name,,,,,,,,,,
Absenteeism,500.0,18.073200,9.341169,0.0,11.800,16.25,22.500,65.8,28.2,1.084104
Air pollution - particulate matter,498.0,9.191165,1.692985,4.9,7.900,9.15,10.300,15.7,9.5,0.182421
Binge drinking,500.0,17.656200,2.684346,9.1,16.000,17.55,19.200,27.4,15.2,-0.915009
Breast cancer deaths,492.0,24.428049,6.056955,11.3,20.300,23.70,27.900,50.6,28.2,0.622747
Cardiovascular disease deaths,494.0,210.440486,58.725131,46.7,171.350,202.75,236.825,515.2,338.4,2.178957
Children in Poverty,500.0,22.625400,10.899536,2.4,14.375,22.15,29.725,60.0,60.0,3.429008
Colorectal cancer deaths,492.0,16.099187,4.207326,4.1,13.500,15.70,18.325,34.3,24.6,2.020479
Dental care,500.0,63.196000,7.546653,42.3,57.600,63.30,68.700,81.8,50.8,-1.642583
Diabetes,500.0,9.997800,2.397196,4.2,8.275,9.80,11.500,21.6,16.6,2.754135


# 2. Find outcomes correlated with absenteeism, binge drinking, and segregation

In [69]:
# transform to pivot table for correlation analysis
df = metrics_totalpop[['city_name', 'metric_name', 'est']]
pivot = pd.pivot_table(df, index='city_name', columns='metric_name', values='est')
pivot.shape


(474, 36)

In [72]:
# Drop all determinants except Absenteeism, Binge Drinking, and Segregation
# include the eleven health outcomes
pivot = pivot[['Absenteeism', 'Breast cancer deaths', 'Cardiovascular disease deaths', 
               'Colorectal cancer deaths', 'Diabetes', 'Frequent mental distress', 
               'High blood pressure', 'Life expectancy', 'Low birthweight', 
               'Obesity', 'Opioid overdose deaths', 'Premature deaths (all causes)', 
               'Binge drinking', 'Neighborhood racial/ethnic segregation']]

# print correlation coefficient matrix
corr = pivot.corr()
corr.style.background_gradient(cmap='coolwarm')

metric_name,Absenteeism,Breast cancer deaths,Cardiovascular disease deaths,Colorectal cancer deaths,Diabetes,Frequent mental distress,High blood pressure,Life expectancy,Low birthweight,Obesity,Opioid overdose deaths,Premature deaths (all causes),Binge drinking,Neighborhood racial/ethnic segregation
metric_name,,,,,,,,,,,,,,
Absenteeism,1,0.0847477,0.141748,0.13306,0.295375,0.397355,0.303157,-0.432124,0.414155,0.388406,0.42859,0.402769,-0.054962,0.29856
Breast cancer deaths,0.0847477,1,0.601428,0.584159,0.211436,0.250258,0.227572,-0.337499,0.339383,0.209285,0.0812774,0.539204,-0.129569,0.189355
Cardiovascular disease deaths,0.141748,0.601428,1,0.719364,0.485293,0.542964,0.426949,-0.5673,0.433209,0.450437,0.159796,0.723895,-0.252892,0.268307
Colorectal cancer deaths,0.13306,0.584159,0.719364,1,0.430219,0.462394,0.367155,-0.518995,0.433328,0.3693,0.157482,0.669625,-0.194426,0.276904
Diabetes,0.295375,0.211436,0.485293,0.430219,1,0.643288,0.856901,-0.666802,0.627521,0.72431,0.318892,0.638199,-0.578695,0.526489
Frequent mental distress,0.397355,0.250258,0.542964,0.462394,0.643288,1,0.557447,-0.781074,0.566533,0.72043,0.458298,0.715763,-0.348813,0.349271
High blood pressure,0.303157,0.227572,0.426949,0.367155,0.856901,0.557447,1,-0.72463,0.738385,0.734432,0.340757,0.646212,-0.469111,0.510187
Life expectancy,-0.432124,-0.337499,-0.5673,-0.518995,-0.666802,-0.781074,-0.72463,1,-0.760371,-0.79897,-0.454029,-0.829765,0.321531,-0.488023
Low birthweight,0.414155,0.339383,0.433209,0.433328,0.627521,0.566533,0.738385,-0.760371,1,0.672716,0.388349,0.689986,-0.284499,0.654301


# 3. Analysis within Flint @ level of census tracts

In [70]:
# read in csv file
tracts = pd.read_csv('CHDB_data_tract_MI v7_1.csv')

# eliminate extraneous columns, keep only tract, metric name, and metric value
tracts = tracts[['tract_code', 'metric_name', 'est']]

#create pivot table
tracts_pivot = pd.pivot_table(tracts, index='tract_code', columns='metric_name', values='est')
tracts_pivot.columns

Index(['Air pollution - particulate matter', 'Binge drinking',
       'Children in Poverty', 'Dental care', 'Diabetes',
       'Frequent mental distress', 'Frequent physical distress',
       'High blood pressure', 'Housing cost, excessive',
       'Housing with potential lead risk', 'Income Inequality',
       'Lead exposure risk index', 'Life expectancy',
       'Limited access to healthy foods', 'Obesity', 'Physical inactivity',
       'Preventive services', 'Racial/ethnic diversity', 'Smoking',
       'Unemployment', 'Uninsured'],
      dtype='object', name='metric_name')

In [71]:
# print correlation coefficient matrix
tracts_corr = tracts_pivot.corr()
tracts_corr.style.background_gradient(cmap='coolwarm')

metric_name,Air pollution - particulate matter,Binge drinking,Children in Poverty,Dental care,Diabetes,Frequent mental distress,Frequent physical distress,High blood pressure,"Housing cost, excessive",Housing with potential lead risk,Income Inequality,Lead exposure risk index,Life expectancy,Limited access to healthy foods,Obesity,Physical inactivity,Preventive services,Racial/ethnic diversity,Smoking,Unemployment,Uninsured
metric_name,,,,,,,,,,,,,,,,,,,,,
Air pollution - particulate matter,1,-0.489896,0.343577,-0.444302,0.515669,0.379198,0.486465,0.583289,0.28418,0.363666,-0.37817,0.351703,-0.414948,-0.252238,0.441747,0.505586,-0.56528,-0.461587,0.454029,0.392778,0.373448
Binge drinking,-0.489896,1,-0.606842,0.661858,-0.910367,-0.504756,-0.750106,-0.905155,-0.5221,-0.408165,0.614161,-0.513275,0.618535,0.131511,-0.746267,-0.77791,0.787205,0.435239,-0.536815,-0.65635,-0.438989
Children in Poverty,0.343577,-0.606842,1,-0.837821,0.712606,0.806446,0.809002,0.590997,0.629965,0.599445,-0.780547,0.763468,-0.677504,-0.186507,0.812042,0.821203,-0.799265,-0.325731,0.80142,0.718407,0.51679
Dental care,-0.444302,0.661858,-0.837821,1,-0.801221,-0.964819,-0.944215,-0.653695,-0.710611,-0.701296,0.907677,-0.864524,0.773077,0.227843,-0.917475,-0.971085,0.95267,0.37424,-0.948001,-0.763671,-0.655245
Diabetes,0.515669,-0.910367,0.712606,-0.801221,1,0.658964,0.88343,0.953011,0.558517,0.5905,-0.75057,0.677806,-0.756158,-0.180095,0.902379,0.891696,-0.860924,-0.545754,0.704139,0.741436,0.522318
Frequent mental distress,0.379198,-0.504756,0.806446,-0.964819,0.658964,1,0.894959,0.498288,0.690018,0.647744,-0.867425,0.832648,-0.714145,-0.217334,0.833602,0.906838,-0.886649,-0.313196,0.965687,0.701024,0.613131
Frequent physical distress,0.486465,-0.750106,0.809002,-0.944215,0.88343,0.894959,1,0.774531,0.608324,0.641371,-0.845434,0.766759,-0.748493,-0.208293,0.906629,0.983674,-0.914783,-0.430105,0.926813,0.737602,0.618208
High blood pressure,0.583289,-0.905155,0.590997,-0.653695,0.953011,0.498288,0.774531,1,0.443701,0.510656,-0.623749,0.555599,-0.715235,-0.173962,0.82436,0.778364,-0.765228,-0.601135,0.582042,0.66302,0.440764
"Housing cost, excessive",0.28418,-0.5221,0.629965,-0.710611,0.558517,0.690018,0.608324,0.443701,1,0.430409,-0.777792,0.663164,-0.619268,-0.227628,0.641684,0.654375,-0.716954,-0.203093,0.620379,0.538845,0.40883
